# 네이버 지식인 크롤링

In [46]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from urllib.parse import quote
from tqdm.notebook import tqdm

In [49]:
chromedriver = 'D:/Workspace/DataAnalysis/05.Crawling/chromedriver.exe'
driver = webdriver.Chrome(chromedriver)

In [51]:
base_url = 'https://kin.naver.com/search/list.naver?query='
url = base_url + quote('여친선물')
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [52]:
lis = soup.select('ul.basic1 > li')
len(lis)

10

In [44]:
li = lis[0]
sub_url = li.select_one('a._searchListTitleAnchor')['href']
sub_url

'https://kin.naver.com/qna/detail.naver?d1id=8&dirId=80101&docId=395035694&qb=7Jes7Lmc7ISg66y8&enc=utf8&section=kin&rank=1&search_sort=0&spq=0'

In [54]:
driver.get(sub_url)
time.sleep(2)

In [57]:
count = int(driver.find_element_by_css_selector('._answerCount.num').text)
answer_page = (count - 1) // 5                          # 5개 답변
print(count, answer_page)                               # answer_page = 더보기 탭 갯수

8 1


In [27]:
for i in range(answer_page):
    driver.find_element_by_id('nextPageButton').click()
    time.sleep(2)

In [64]:
divs = soup.select('div.answer-content__item._contentWrap._answer')
len(divs)

5

In [65]:
div = divs[0]
ps = div.select('p.se-text-paragraph')
for p in ps:
    print(p.get_text())

혹시 여친분이 생리통이 있으시다면..여친선물로 복부온열기 어떠세요?
저 생리통 너무 심해서 이번에 남자친구가 복부온열기 선물줬는데..
뭔가 감동이더라구요. 절 많이 생각해서 준 선물 같이 느껴지더라구요.
하고있으면 배가 뜨듯하고 진동마사지도 해줘서 생리통 거의 안느껴지고 좋아요!!
​
제가 쓰는건 배뜨끈 복부온열기이예요. 아래에 링크알려드릴게요.
좋은 여친선물이 되셨으면 좋겠네요.


In [69]:
divs[6].select('p.se-text-paragraph')[0]

<p class="se-text-paragraph se-text-paragraph-align-" id="SE-42015bc3-7bef-455d-a76d-2bfdf3237560" style=""><span class="se-fs- se-ff-" id="SE-c12a6038-91dd-45db-9fa5-68e6019b3e30" style="">안녕하세요, 공연기획사 파릇입니다!</span></p>

In [68]:
driver.back()
time.sleep(2)

In [73]:
answer_list = []
for li in lis:
    sub_url = li.select_one('a._searchListTitleAnchor')['href']
    driver.get(sub_url)                 # 서브페이지 이동
    time.sleep(2)

    count = int(driver.find_element_by_css_selector('._answerCount.num').text)
    answer_page = (count - 1) // 5
    print(count, end=', ')                  # 답변갯수 구함         # count = answer 갯수
    for i in range(answer_page):
        driver.find_element_by_id('nextPageButton').click()
        time.sleep(2)                       # 반복해서 answer 페이지에 모든 갯수를 화면에 나오게 함

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    divs = soup.select('div.answer-content__item._contentWrap._answer')
    print(len(divs))
    for div in divs:
        ps = div.select('p.se-text-paragraph')
        paragraph = ''                                  # 데이터가 한 줄이아니라 여러줄로 나와야함
        for p in ps:
            text = p.get_text()
            if text == ' ' or text == '\u200b':
                continue
            paragraph += text + '\n'                    # 데이터가 한 줄이아니라 여러줄로 나와야함2
            answer_list.append(text)

    driver.back()
    time.sleep(2)

8, 8
10, 10
7, 7
12, 12
14, 14
9, 9
1, 1
4, 4
1, 1
8, 8


In [75]:
answer_list[100:110]

['네이버 선물하기로 카톡 선물하기처럼 보낼 수 있어요!!',
 '구경만이라도 해보세요:)',
 '채택도... 부탁드립니다',
 'https://smartstore.naver.com/ppottuppttu',
 '안녕하세요.^^',
 '여자친구분 선물로 팔찌로 추천해드릴께요.',
 '크로노스',
 '크로노스 CRJ2018B-RG THICK 하트 팔찌 입니다.',
 '스텐레스 스틸 재질로 변질이 적으며 한쪽과 반대쪽을 다르게 디자인되어있으며',
 '하트 달랑이가 돋보이는 팔찌입니다.']

## 1 ~ 10 페이지 데이터 한꺼번에 하기

In [78]:
answer_list = []
for page in tqdm(range(1,11)):
    #print('page', page)
    url = f"{base_url}{quote('여친선물')}&page={page}"
    driver.get(url)
    time.sleep(2)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    lis = soup.select('ul.basic1 > li')

    for li in lis:
        sub_url = li.select_one('a._searchListTitleAnchor')['href']
        driver.get(sub_url)
        time.sleep(2)

        count = int(driver.find_element_by_css_selector('._answerCount.num').text)
        answer_page = (count - 1) // 5
        #print(count, end=', ')
        for i in range(answer_page):
            driver.find_element_by_id('nextPageButton').click()
            time.sleep(2)

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        divs = soup.select('div.answer-content__item._contentWrap._answer')
        #print(len(divs))
        for div in divs:
            ps = div.select('p.se-text-paragraph')
            paragraph = ''
            for p in ps:
                text = p.get_text()
                if text == ' ' or text == '\u200b':
                    continue
                paragraph += text + '\n'
            answer_list.append(paragraph)

        driver.back()
        time.sleep(2)

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
driver.close()

In [79]:
len(answer_list)

679

In [80]:
answer_list[0]

'혹시 여친분이 생리통이 있으시다면..여친선물로 복부온열기 어떠세요?\n저 생리통 너무 심해서 이번에 남자친구가 복부온열기 선물줬는데..\n뭔가 감동이더라구요. 절 많이 생각해서 준 선물 같이 느껴지더라구요.\n하고있으면 배가 뜨듯하고 진동마사지도 해줘서 생리통 거의 안느껴지고 좋아요!!\n제가 쓰는건 배뜨끈 복부온열기이예요. 아래에 링크알려드릴게요.\n좋은 여친선물이 되셨으면 좋겠네요.\n'

In [86]:
with open('data/new_gift.txt', 'w', encoding='utf-8') as fp:
    for answer in answer_list:
        if answer[:8] == 'https: ':
            continue
        fp.write(answer+'\n')

In [82]:
driver.close()